In [ ]:
!pip install git+https://github.com/facebookresearch/fvcore.git


  Cloning https://github.com/facebookresearch/fvcore.git to /tmp/pip-req-build-ikjl_vef
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fvcore.git /tmp/pip-req-build-ikjl_vef
  Resolved https://github.com/facebookresearch/fvcore.git to commit 0f2b23b6f93e36041d9a74764ee824541cf0a0e5
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.6-py3-none-any.whl size=65268 sha256=04e760274b838eee17d1f9649cf87b693f5b3160851b2446829f47b558a20b8c
  Stored in directory: /tmp/pip-ephem-wheel-cache-fe8gxk65/wheels/8f/cb/6a/3b7ac0e01781855ca3d1417ebf9e15e20d5b7fe37ab063aa50
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31530 sha256=a0b0c5763e95ec730e546595ffaa4ff1023ae183f4b07f95ed3eb213775fe58f
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e47

In [ ]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings(action='ignore')





In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [ ]:
# dataset_path = os.listdir("/content/drive/MyDrive/keb/data")
# if ".DS_Store" in dataset_path:
#   dataset_path.remove(".DS_Store")
# print(dataset_path)


# rooms = []

# for item in dataset_path:
#   all_rooms = os.listdir("/content/drive/MyDrive/keb/data"+"/"+item) #각 폴더에 들어갈 데이터의 이름들 가져오기.
#   print("items : ",len(all_rooms))
#   for room in all_rooms:
#     #tag,이름
#     rooms.append((item, str("/content/drive/MyDrive/keb/data"+"/"+item)+"/"+room))


# train_df = pd.DataFrame(data=rooms, columns =["tag", "video_name"])


# df = train_df.loc[:,["video_name", "tag"]]
# df.to_csv("data_test.csv")

# data = pd.read_csv("data_test.csv")
# print(data.head())
# print(data.tail())
# print("")
# data["tag"] = data["tag"].map({"폭력": 1, "정상": 0})


# data = data[["video_name","tag"]]
# print(data.tail())

# #train, valid, test로 만들어야함.

# train_ = data.sample(frac=0.6, random_state=42)
# tmp = data.drop(train_.index)
# valid = tmp.sample(frac=0.5, random_state=42)
# test = tmp.drop(valid.index)


# train_.to_csv("/content/drive/MyDrive/keb/train_test.csv")
# valid.to_csv("/content/drive/MyDrive/keb/valid_test.csv")
# test.to_csv("/content/drive/MyDrive/keb/test_test.csv")

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/keb/train_test.csv")
test = pd.read_csv("/content/drive/MyDrive/keb/test_test.csv")
valid = pd.read_csv("/content/drive/MyDrive/keb/valid_test.csv")

In [ ]:
print(len(train))
print(len(valid))
print(len(test))

568
189
189


In [ ]:
train.head()

,Unnamed: 0,video_name,tag
0,327,/content/drive/MyDrive/keb/data/폭력/C_3_13_...,1
1,30,/content/drive/MyDrive/keb/data/폭력/C_3_13_...,1
2,820,/content/drive/MyDrive/keb/data/정상/C_2_4_7...,0
3,404,/content/drive/MyDrive/keb/data/폭력/C_3_13_...,1
4,76,/content/drive/MyDrive/keb/data/폭력/C_3_13_...,1


In [ ]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [ ]:
class CustomDataset(Dataset):
  def __init__(self, video_path_list,label_list):
    self.video_path_list = video_path_list
    self.label_list = label_list

  def __getitem__(self, index):
    frames = self.get_video(self.video_path_list[index])
    low_frames = frames[:,[i*4+2 for i in range(12)],:,:]

    if self.label_list is not None:
      label = self.label_list[index]
      return low_frames, frames, label
    else:
      return low_frames, frames

  def __len__(self):
    return len(self.video_path_list)

  def get_video(self, path):
    frames = []
    cap = cv2.VideoCapture(path)
    for _ in range(47):
      _, img = cap.read()

      if img is not None:
        img = cv2.resize(img, (256,256))
        img = img/255
        frames.append(img)

    cap.release()


    return torch.FloatTensor(np.array(frames)).permute(3,0,1,2)




In [ ]:
train_dataset = CustomDataset(train["video_name"].values, train["tag"].values)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0)

val_dataset = CustomDataset(valid["video_name"].values, valid["tag"].values)
val_loader = DataLoader(val_dataset, batch_size =4, shuffle=False, num_workers=0)

In [ ]:
# model_name = "slowfast_r50"
# slowfast_r50_layers = slowfast_r50.blocks

In [ ]:
class Slowfast(nn.Module):
  def __init__(self, num_classes=2, model_name = "slowfast_r50"):
    super(Slowfast,self).__init__()
    self.backbone = torch.hub.load("facebookresearch/pytorchvideo", model=model_name,pretrained=True)

    output_dim = 400 #slowfast의 마지막 출력값.

    self.classifier = nn.Sequential(
        nn.Linear(output_dim, 256),
        nn.ReLU(),
        nn.Linear(256, num_classes)
    )

  def forward(self, x):
    print("x1 : ",x[0].shape)
    print("x1 : ",x[1].shape)
    x = self.backbone(x)
    print("x2 : ",x[0].shape)

    x = self.classifier(x)

    return x

In [ ]:
model_name = "slowfast_r50"
slowfast_r50 = torch.hub.load("facebookresearch/pytorchvideo", model=model_name,pretrained=True)
slowfast_r50_layers = slowfast_r50.blocks

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [ ]:
class SlowFast(nn.Module):
    def __init__(self,img_size=256):
        super(SlowFast, self).__init__()
        self.pool_size = img_size//32-1

        self.layer_1 = slowfast_r50_layers[0]
        self.layer_2 = slowfast_r50_layers[1]
        self.layer_3 = slowfast_r50_layers[2]
        self.layer_4 = slowfast_r50_layers[3]
        self.layer_5 = slowfast_r50_layers[4]
        self.fast_pool = nn.AvgPool3d(kernel_size=(12, self.pool_size, self.pool_size), stride=(1, 1, 1), padding=(0, 0, 0))
        self.slow_pool = nn.AvgPool3d(kernel_size=(48, self.pool_size, self.pool_size), stride=(1, 1, 1), padding=(0, 0, 0))

        self.avgpool = nn.AdaptiveAvgPool3d(1)
        self.fc = nn.Linear(2304,2) #일단은

        for para in self.layer_1.parameters():
           para.requires_grad = False
        for para in self.layer_2.parameters():
           para.requires_grad = False

    def forward(self, x):
        batch_size = x[0].size(0)
        x = self.layer_1(x)
        x = self.layer_2(x)
        x = self.layer_3(x)
        x = self.layer_4(x)
        x = self.layer_5(x)

        x_1 = self.fast_pool(x[0])
        x_2 = self.slow_pool(x[1])
        x = torch.concat([x_1,x_2],axis=1)
        x = self.avgpool(x).view(batch_size,-1)

        x = self.fc(x)

        return x

In [ ]:
def validation(model, criterion, val_loader, device):
  model.eval()
  val_loss = []
  preds, trues = [],[]

  with torch.no_grad():
    # for videos, labels in tqdm(iter(val_loader)):
    for slow_videos, videos, labels in tqdm(iter(val_loader)):
      slow_videos = slow_videos.to(device)
      videos = videos.to(device)
      labels = labels.to(device)
      print(labels)

      logit = model([slow_videos,videos])

      loss = criterion(logit, labels)
      val_loss.append(loss.item())

      preds +=logit.argmax(1).detach().cpu().numpy().tolist()
      trues += labels.detach().cpu().numpy().tolist()

    _val_loss = np.mean(val_loss)

  _val_score = f1_score(trues, preds, average="macro")
  return _val_loss, _val_score

In [ ]:
def train_(model, optimizer, train_loader, val_loader, schedule, device):
  model.to(device)
  criterion = nn.CrossEntropyLoss().to(device)

  best_val_score = 0
  best_model = None

  for epoch in range(1,11): #테스트 용이니까.
    model.train()
    train_loss=[]
    for slow_videos, videos, labels in tqdm(train_loader):
      slow_videos = slow_videos.to(device)
      videos = videos.to(device)
      labels = labels.to(device)

      optimizer.zero_grad()
      output = model([slow_videos,videos])

      loss = criterion(output, labels)

      loss.backward()
      optimizer.step()

      train_loss.append(loss.item())

    _val_loss, _val_score = validation(model, criterion, val_loader, device)
    _train_loss = np.mean(train_loss)
    print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 : [{_val_score:.5f}]')

    if schedule is not None:
      schedule.step(_val_score)

    if best_val_score < _val_score:
      best_val_score = _val_score
      best_model = model

  return best_model

In [ ]:
model = SlowFast()
print(model)

SlowFast(
  (layer_1): MultiPathWayWithFuse(
    (multipathway_blocks): ModuleList(
      (0): ResNetBasicStem(
        (conv): Conv3d(3, 64, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
        (norm): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
        (pool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=[0, 1, 1], dilation=1, ceil_mode=False)
      )
      (1): ResNetBasicStem(
        (conv): Conv3d(3, 8, kernel_size=(5, 7, 7), stride=(1, 2, 2), padding=(2, 3, 3), bias=False)
        (norm): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
        (pool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=[0, 1, 1], dilation=1, ceil_mode=False)
      )
    )
    (multipathway_fusion): FuseFastToSlow(
      (conv_fast_to_slow): Conv3d(8, 16, kernel_size=(7, 1, 1), stride=(4, 1, 1), padding=(3, 0, 0), bias=False)


In [ ]:


model.eval()
# optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
optimizer = torch.optim.AdamW(params = model.parameters(), lr =0.001, weight_decay=0.05)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[12], gamma=0.1)

# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", factor=0.5, patience=2, threshold_mode="abs",
                                                      # min_lr=1e-8, verbose=True)
infer_model = train_(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/142 [00:00<?, ?it/s]

RuntimeError: ignored